In [4]:
import pandas as pd
import copy

# Load data
gg_form = pd.read_csv('test data for schedule code.csv')
gg_form.fillna('-', inplace=True)  # Fill NaN values with '-'

# Initialize dictionary to store available times
available_time = {column: {} for column in gg_form.columns if column not in ['ประทับเวลา', 'ชื่อเล่น (nickname)']}

times = [
    '9.00-9.30', '9.30-10.00', '10.00-10.30', '10.30-11.00', '11.00-11.30', '11.30-12.00',
    '12.00-12.30', '12.30-13.00', '13.00-13.30', '13.30-14.00', '14.00-14.30', '14.30-15.00',
    '15.00-15.30', '15.30-16.00', '16.00-16.30', '16.30-17.00', '17.00-17.30', '17.30-18.00',
    '18.00-18.30', '18.30-19.00', '19.00-19.30', '19.30-20.00', '20.00-20.30'
]

# Initialize available time slots
def initialize_time_slots():
    return {time: [] for time in times}

for day in available_time:
    available_time[day] = initialize_time_slots()

# Process data to populate available_time
gg_form_loop = gg_form.copy()
for _, row in gg_form_loop.iterrows():
    nickname = row['ชื่อเล่น (nickname)']
    for day in available_time:
        for time in times:
            if row[day] == time:
                available_time[day][time].append(nickname)

# Create nickname dictionary
nickname_dict = {}
for _, row in gg_form_loop.iterrows():
    nickname = row['ชื่อเล่น (nickname)']
    nickname_dict[nickname] = {
        column: row[column].split(', ') if isinstance(row[column], str) and row[column] != '-' else []
        for column in available_time
    }

# Duplicate available_time to track members per time slot
available_time_member = copy.deepcopy(available_time)
for nickname, schedule in nickname_dict.items():
    for day, times_list in schedule.items():
        for time in times_list:
            if time in available_time_member[day]:
                available_time_member[day][time].append(nickname)

# Define songs and members
song_dict_count = {
    'Song 1': 0, 'Song 2': 0, 'Song 3': 0,
    'Song 4': 0, 'Song 5': 0, 'Song 6': 0
}

song_member_dict = {
    'Song 1': ['Member 1', 'Member 2', 'Member 3', 'Member4', 'Member 5', 'Member 6', 'Member 7'],
    'Song 2': ['Member 8', 'Member 7', 'Member 9', 'Member 10', 'Member 11'],
    'Song 3': ['Member 12', 'Member 13', 'Member 4', 'Member 14', 'Member 15', 'Member 10'],
    'Song 4': ['Member 2', 'Member 12', 'Member 19', 'Member 13', 'Member 16', 'Member 17', 'Member 10'],
    'Song 5': ['Member 18', 'Member 4', 'Member 7', 'Member 15', 'Member 11'],
    'Song 6': ['Member 12', 'Member 19', 'Member 8', 'Member 20', 'Member 21', 'Member 11']
}

# Initialize time_song_member_count
time_song_member_count = {
    day: {
        time: {song: {'count': 0, 'members': []} for song in song_dict_count}
        for time in times
    }
    for day in available_time
}

# Populate time_song_member_count
for day in available_time:
    for time in times:
        for song, members in song_member_dict.items():
            for member in available_time_member[day][time]:
                if member in members:
                    time_song_member_count[day][time][song]['count'] += 1
                    time_song_member_count[day][time][song]['members'].append(member)

# Prepare spreadsheet data
spreadsheet_data = []
for day, time_slots in time_song_member_count.items():
    for time, song_data in time_slots.items():
        row_data = {'Date': day, 'Time': time}
        for song, details in song_data.items():
            row_data[song] = f"{details['count']} members: {', '.join(details['members'])}" if details['members'] else "0 members"
        spreadsheet_data.append(row_data)

# Create DataFrame and export to CSV
df = pd.DataFrame(spreadsheet_data)
df.to_csv('calendar_data.csv', index=False)